# Imports

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import iqr, entropy
import os, re, glob, json
import pandas as pd
import numpy as np
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from constants import SCORE_COLS, XY_COLS
from preprocessing_utils import interpolate_track

# Reading the files

In [2]:
FOLDER = r"C:\Users\ioana\OneDrive\Desktop\annotations"     
N_FILES = 187 #187 total                              
name_re = re.compile(
    r"(?P<date>\d{4}_\d{2}_\d{2})_"      # 2021_07_26
    r"\d+_"                              # recording number we ignore (833180)
    r"(?P<participant>\d+)_"             # 201
    r"cam(?P<cam>\d+)_vid\d+\.json$"     # cam3
)

In [3]:
rows = []
for path in sorted(glob.glob(os.path.join(FOLDER, "*.json")))[:N_FILES]:
    m = name_re.search(os.path.basename(path))
    if not m:
        print(f" Skipped (name format?): {path}")
        continue
    
    date       = pd.to_datetime(m.group("date"), format="%Y_%m_%d")
    participant= int(m.group("participant"))
    camera     = int(m.group("cam"))
    
    with open(path, "r") as f:
        frames = json.load(f)          # top level = list of frames
    
    for frame in frames:
        fid = frame["frame_id"]
        for inst_i, inst in enumerate(frame["instances"]):
            kps     = inst["keypoints"]        # list[[x,y], …] length 17
            kp_s    = inst["keypoint_scores"]  # list[score]  length 17
            
            # Flatten keypoints so every (x,y,score) gets its own column
            record = {
                "date"          : date,
                "camera"        : camera,
                "participant_id": participant,
                "frame_id"      : fid,
                "instance_idx"  : inst_i,
            }
            for i, (xy, sc) in enumerate(zip(kps, kp_s)):
                record[f"x{i}"]     = xy[0]
                record[f"y{i}"]     = xy[1]
                record[f"score{i}"] = sc
            rows.append(record)

df = pd.DataFrame(rows)

In [4]:
df.head()

,date,camera,participant_id,frame_id,instance_idx,x0,y0,score0,x1,y1,...,score13,x14,y14,score14,x15,y15,score15,x16,y16,score16
0,2021-07-26,3,201,0,0,361.027242,667.847929,0.920992,339.677817,678.178145,...,0.985141,790.259357,923.892419,0.950637,892.392309,667.571038,0.969989,882.349755,894.508950,0.910822
1,2021-07-26,3,201,0,1,804.504721,1868.637342,0.257770,810.447079,1866.809957,...,0.748712,1074.342156,1617.537629,0.584149,1018.423242,1866.561995,0.088156,1064.025339,1629.099040,0.266310
2,2021-07-26,3,201,0,2,822.541713,1871.110857,0.596883,841.598718,1877.882258,...,0.242272,321.221966,1871.953478,0.390355,144.518203,1896.295941,0.961632,280.470757,1905.495459,0.915473
3,2021-07-26,3,201,0,3,806.213967,1892.929661,0.100071,795.759942,1885.918862,...,0.694224,839.971939,1859.068609,0.642309,823.505486,1889.627250,0.179076,840.940509,1904.022024,0.143290
4,2021-07-26,3,201,1,0,360.278843,668.007866,0.931771,338.452149,679.163287,...,0.982250,793.741293,925.141581,0.957903,894.766242,668.896053,0.952610,885.131452,890.948337,0.933250


In [5]:
# 0) compute mean score -------------------------------------------------
score_cols = [c for c in df.columns if c.startswith("score")]
df["mean_score"] = df[score_cols].mean(axis=1)

# 1) choose the highest-quality instance per unique frame --------------
best_idx = (
    df.groupby(["date", "participant_id", "camera", "frame_id"])
      ["mean_score"]
      .idxmax()                      # row-label of the top-scoring instance
)

keepers = (
    df.loc[best_idx]                 # pull those rows out of the big DF
      .sort_values(                  # 2) put them back in chronological order
          ["date", "participant_id", "camera", "frame_id"],
          ascending=[True, True, True, True]
      )
      .reset_index(drop=True)
)

print(keepers.head())

        date  camera  participant_id  frame_id  instance_idx          x0  \
0 2021-07-26       3             201         0             0  361.027242   
1 2021-07-26       3             201         1             0  360.278843   
2 2021-07-26       3             201         2             0  361.083892   
3 2021-07-26       3             201         3             0  362.031812   
4 2021-07-26       3             201         4             0  360.686736   

           y0    score0          x1          y1  ...         x14         y14  \
0  667.847929  0.920992  339.677817  678.178145  ...  790.259357  923.892419   
1  668.007866  0.931771  338.452149  679.163287  ...  793.741293  925.141581   
2  666.328148  0.929269  340.329540  676.496263  ...  793.771664  925.053525   
3  665.457730  0.929338  340.455224  676.332937  ...  794.021854  924.631746   
4  665.324786  0.919933  339.106871  676.612846  ...  796.034762  924.224804   

    score14         x15         y15   score15         x16     

In [6]:
summary = (
    keepers
      .groupby(["date", "participant_id"])
      .agg(n_frames=("frame_id", "nunique"))   # count unique frames
      .reset_index()
      .sort_values(["date", "participant_id"])
)

print("\nFrames kept per participant & date:")
print(summary.to_string(index=False))


Frames kept per participant & date:
      date  participant_id  n_frames
2021-07-26             201      8410
2021-08-17             202      3687
2021-08-19             201      7481
2021-09-09             202      9206
2021-09-15             203      7501
2021-09-16             201      7486
2021-10-01             202      4060
2021-10-06             203      7717
2021-10-07             201      7529
2021-11-05             203      7493
2021-11-11             201      7538
2021-12-10             202      7831
2021-12-15             204      7525
2021-12-16             201      7658
2022-01-12             202      7652
2022-01-12             204      7616
2022-02-04             203      7573
2022-02-14             207      7574
2022-02-16             204        45
2022-03-16             204      7950
2022-03-18             207      7513
2022-04-11             207      7471
2022-04-19             204      7868
2022-05-12             211      4375
2022-05-17             204      7752
2

In [7]:
keepers_interp = (
    keepers
      .groupby(["participant_id", "date", "camera"], group_keys=False)
      .apply(interpolate_track)
)

# ── everything below stays the same ────────────────────────────
MIN_FRAMES = 1_000

counts = (keepers_interp
          .groupby(["participant_id", "date", "camera"])
          .size()
          .rename("n_frames"))

good_sessions = counts[counts >= MIN_FRAMES].index

keepers_long = (
    keepers_interp
      .set_index(["participant_id", "date", "camera"])
      .loc[good_sessions]
      .reset_index()
)

C:\Users\ioana\AppData\Local\Temp\ipykernel_4284\3455364051.py:4: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(interpolate_track)


In [8]:
keepers_long = keepers_long.dropna(subset=XY_COLS)
keepers_long

,participant_id,date,camera,frame_id,instance_idx,x0,y0,score0,x1,y1,...,x14,y14,score14,x15,y15,score15,x16,y16,score16,mean_score
0,201,2021-07-26,3,0,0,361.027242,667.847929,0.920992,339.677817,678.178145,...,790.259357,923.892419,0.950637,892.392309,667.571038,0.969989,882.349755,894.508950,0.910822,0.907676
1,201,2021-07-26,3,1,0,360.278843,668.007866,0.931771,338.452149,679.163287,...,793.741293,925.141581,0.957903,894.766242,668.896053,0.952610,885.131452,890.948337,0.933250,0.912949
2,201,2021-07-26,3,2,0,361.083892,666.328148,0.929269,340.329540,676.496263,...,793.771664,925.053525,0.951839,895.143201,666.911500,0.943813,880.495490,891.889313,0.911516,0.909255
3,201,2021-07-26,3,3,0,362.031812,665.457730,0.929338,340.455224,676.332937,...,794.021854,924.631746,0.952851,896.068243,667.004500,0.939086,881.935661,889.416794,0.923034,0.908262
4,201,2021-07-26,3,4,0,360.686736,665.324786,0.919933,339.106871,676.612846,...,796.034762,924.224804,0.925136,899.272541,669.322768,0.939291,878.945773,889.556496,0.911612,0.905861
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1278922,252,2024-06-06,3,7430,1,873.177611,709.993579,0.950272,841.832781,728.368880,...,877.052384,404.132315,0.828778,764.907566,284.122909,0.704885,790.085899,308.952887,0.864322,0.886717
1278923,252,2024-06-06,3,7431,0,873.600185,707.297233,0.864843,845.771279,724.564801,...,870.562807,397.428835,0.817476,763.427691,286.100351,0.711817,788.626592,306.405303,0.855806,0.865212
1278924,252,2024-06-06,3,7432,0,876.123413,707.999518,0.943470,846.210581,728.927971,...,873.291293,400.414112,0.816270,762.674478,289.751913,0.747307,791.160727,305.532321,0.835189,0.887962
1278925,252,2024-06-06,3,7433,0,874.728845,706.790113,0.918305,847.001123,728.145727,...,871.768114,399.249226,0.807922,763.139589,286.869361,0.760217,791.793849,303.570067,0.812956,0.880797


In [9]:
print(f"Dropped {len(keepers) - len(keepers_long)} rows "
      f"from {len(counts) - len(good_sessions)} recordings "
      f"(< {MIN_FRAMES} frames)")

Dropped 6181 rows from 5 recordings (< 1000 frames)


In [10]:
keepers_long.to_csv("infant_keypoints__187files.csv")

# Feature Extraction

 From the pose-tracked video data, compute a reduced feature set grouped into six terms (we use the existing feature code for this):

- Posture – median joint angles, median positions (angles for elbows, knees, positions for writs, elbows, ankles, knees). 
- Symmetry – cross-correlation between left/right limbs (for writs, elbows, ankles, knees). .
- Smoothness – RMS jerk, IQR velocity. (all key points) 
- Variability/complexity – entropy measures of position and angle. (angles for elbows, knees, positions - wrists, elbows, ankles, knees) 
- Range – IQR position or IQR joint angles. (all key points, joint angles --> elbows, knees) 
- Effort – median velocity, IQR acceleration.(all key points)

Keep features consistent across all infants (same joints, same units, same time windowing).

Use a fixed sliding window length (e.g., 3–5 seconds) with 50% overlap.
For each window, compute the six IOC term values (one scalar per term).

In [11]:
from feature_computation import vel, acc, median_position, median_velocity, signal_entropy, cross_corr
from feature_computation import angle, angular_vel, angular_acc, rms_jerk

In [12]:
from scipy.interpolate import UnivariateSpline

def spline_smoother(xs, ys, fr, s_frac=0.5, k=3):
    """
    Fit cubic smoothing splines to x(t), y(t) over frames fr.
    s = s_frac * N is a simple, tunable smoothing level (N = #samples).
    Increase s_frac → more smoothing.
    """
    fr = np.asarray(fr, dtype=float)

    m = np.isfinite(fr) & np.isfinite(xs)
    sx = UnivariateSpline(fr[m], np.asarray(xs)[m], k=k, s=s_frac * m.sum())
    m = np.isfinite(fr) & np.isfinite(ys)
    sy = UnivariateSpline(fr[m], np.asarray(ys)[m], k=k, s=s_frac * m.sum())

    return sx(fr), sy(fr)

def smooth_session(sess_df, smoother_fn):
    """
    Apply a coordinate smoother to an entire session (one participant/date/camera).
    Returns a dataframe with the same keys + smoothed x*, y* columns.
    """
    fr = sess_df["frame_id"].to_numpy(dtype=float)
    out = {"frame_id": fr}
    for k in range(17):
        xs = sess_df[f"x{k}"].to_numpy()
        ys = sess_df[f"y{k}"].to_numpy()
        sx, sy = smoother_fn(xs, ys, fr)
        out[f"x{k}"] = sx
        out[f"y{k}"] = sy

    sm = pd.DataFrame(out, index=sess_df.index)
    meta = sess_df[["participant_id","date","camera"]]
    return pd.concat([meta.reset_index(drop=True), sm.reset_index(drop=True)], axis=1)

In [ ]:
# ankles = [15, 16]  # left and right ankle key‑points (left, right)
# wrists = [9, 10]  # left and right wrist key‑points
# knees = [13, 14]  # left and right knee key‑points
# elbows = [7, 8]  # left and right elbow key‑points

In [13]:
def reduced_features(df, fps=30.0):
    dt = 1.0 / fps
    feats = {}

    # === Extract coordinates ===
    # Wrists
    lw = df[["x9","y9"]].to_numpy()
    rw = df[["x10","y10"]].to_numpy()
    # Elbows
    le = df[["x7","y7"]].to_numpy()
    re = df[["x8","y8"]].to_numpy()
    # Shoulders (for elbow angles)
    ls = df[["x5","y5"]].to_numpy()
    rs = df[["x6","y6"]].to_numpy()
    # Knees
    lk = df[["x13","y13"]].to_numpy()
    rk = df[["x14","y14"]].to_numpy()
    # Hips (for knee angles)
    lh = df[["x11","y11"]].to_numpy()
    rh = df[["x12","y12"]].to_numpy()
    # Ankles
    la = df[["x15","y15"]].to_numpy()
    ra = df[["x16","y16"]].to_numpy()

    # === Joint angles ===
    theta_el_l = angle(ls, le, lw)
    theta_el_r = angle(rs, re, rw)
    theta_kn_l = angle(lh, lk, la)
    theta_kn_r = angle(rh, rk, ra)

    # === 1. Posture ===
    feats["posture_median_angle_elbows"] = np.median(np.hstack([theta_el_l, theta_el_r]))
    feats["posture_median_angle_knees"]  = np.median(np.hstack([theta_kn_l, theta_kn_r]))
    feats["posture_median_pos_wrists"]   = np.median(np.vstack([lw, rw]))
    feats["posture_median_pos_elbows"]   = np.median(np.vstack([le, re]))
    feats["posture_median_pos_ankles"]   = np.median(np.vstack([la, ra]))
    feats["posture_median_pos_knees"]    = np.median(np.vstack([lk, rk]))

    # === 2. Symmetry ===
    feats["sym_wrists"] = np.corrcoef(lw.flatten(), rw.flatten())[0,1]
    feats["sym_elbows"] = np.corrcoef(theta_el_l, theta_el_r)[0,1]
    feats["sym_knees"]  = np.corrcoef(theta_kn_l, theta_kn_r)[0,1]
    feats["sym_ankles"] = np.corrcoef(la.flatten(), ra.flatten())[0,1]

    # === 3. Smoothness ===
    rms_all = []
    iqr_vel_all = []
    for k in range(17):  # all keypoints
        xs, ys = df[f"x{k}"].to_numpy(), df[f"y{k}"].to_numpy()
        rms_all.append(rms_jerk(xs, dt))
        rms_all.append(rms_jerk(ys, dt))
        vmag = np.linalg.norm(np.stack([vel(xs,dt), vel(ys,dt)]), axis=0)
        iqr_vel_all.append(iqr(vmag))
    feats["smooth_rms_jerk"] = np.mean(rms_all)
    feats["smooth_iqr_vel"]  = np.mean(iqr_vel_all)

    # === 4. Variability / Complexity ===
    # positions
    feats["var_entropy_wrists"] = entropy(np.histogram(np.hstack([lw.flatten(), rw.flatten()]), bins=32, density=True)[0]+1e-12)
    feats["var_entropy_elbows"] = entropy(np.histogram(np.hstack([le.flatten(), re.flatten()]), bins=32, density=True)[0]+1e-12)
    feats["var_entropy_ankles"] = entropy(np.histogram(np.hstack([la.flatten(), ra.flatten()]), bins=32, density=True)[0]+1e-12)
    feats["var_entropy_knees"]  = entropy(np.histogram(np.hstack([lk.flatten(), rk.flatten()]), bins=32, density=True)[0]+1e-12)
    # angles
    feats["var_entropy_angles_elbows"] = entropy(np.histogram(np.hstack([theta_el_l, theta_el_r]), bins=32, density=True)[0]+1e-12)
    feats["var_entropy_angles_knees"]  = entropy(np.histogram(np.hstack([theta_kn_l, theta_kn_r]), bins=32, density=True)[0]+1e-12)

    # === 5. Range ===
    iqr_pos_all = []
    for k in range(17):
        xs, ys = df[f"x{k}"].to_numpy(), df[f"y{k}"].to_numpy()
        iqr_pos_all.append(iqr(xs))
        iqr_pos_all.append(iqr(ys))
    feats["range_iqr_pos"] = np.mean(iqr_pos_all)
    feats["range_iqr_angle_elbows"] = np.mean([iqr(theta_el_l), iqr(theta_el_r)])
    feats["range_iqr_angle_knees"]  = np.mean([iqr(theta_kn_l), iqr(theta_kn_r)])

    # === 6. Effort ===
    med_vel_all = []
    iqr_acc_all = []
    for k in range(17):
        xs, ys = df[f"x{k}"].to_numpy(), df[f"y{k}"].to_numpy()
        vmag = np.linalg.norm(np.stack([vel(xs,dt), vel(ys,dt)]), axis=0)
        amag = np.linalg.norm(np.stack([acc(xs,dt), acc(ys,dt)]), axis=0)
        med_vel_all.append(np.median(vmag))
        iqr_acc_all.append(iqr(amag))
    feats["effort_median_vel"] = np.mean(med_vel_all)
    feats["effort_iqr_acc"]    = np.mean(iqr_acc_all)

    return feats

In [14]:
def windowed_features(sess_df, fps=30.0, win_sec=4, step_sec=2,
                      smoother_fn=None, smoother_name="raw"):
    """
    Compute reduced features per sliding window for one session.
    If smoother_fn is given, the session is smoothed globally first.
    """
    # smooth globally (recommended)
    if smoother_fn is not None:
        sess_df = smooth_session(sess_df, smoother_fn)

    win_len = int(win_sec * fps)
    step    = int(step_sec * fps)

    frames = sess_df["frame_id"].to_numpy()
    min_f, max_f = frames.min(), frames.max()

    results = []
    start = min_f
    while start + win_len <= max_f:
        stop = start + win_len
        win_df = sess_df[(sess_df["frame_id"] >= start) &
                         (sess_df["frame_id"] <  stop)]

        if len(win_df) < win_len:
            start += step
            continue

        feats = reduced_features(win_df, fps=fps)   # <-- your function

        # identifiers
        first = sess_df.iloc[0]
        feats.update({
            "participant": first["participant_id"],
            "date": first["date"],
            "camera": first["camera"],
            "win_start": start,
            "win_stop":  stop,
            "n_frames":  len(win_df),
            "filter":    smoother_name,
        })
        results.append(feats)
        start += step

    return pd.DataFrame(results)

In [15]:
# === Apply to all sessions ===
spline = lambda x, y, fr: spline_smoother(x, y, fr, s_frac=0.5, k=3)

all_results = []
for (pid, date, cam), sess_df in keepers_long.groupby(["participant_id","date","camera"], group_keys=False):
    feats_df = windowed_features(sess_df, fps=30, win_sec=4, step_sec=2,
                                 smoother_fn=spline, smoother_name="spline")
    all_results.append(feats_df)

features_windowed = pd.concat(all_results, ignore_index=True)

C:\Users\ioana\AppData\Local\Temp\ipykernel_4284\907160033.py:12: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  sx = UnivariateSpline(fr[m], np.asarray(xs)[m], k=k, s=s_frac * m.sum())
C:\Users\ioana\AppData\Local\Temp\ipykernel_4284\907160033.py:12: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  sx = UnivariateSpline(fr[m], np.asarray(xs)[m], k=k, s=s_frac * m.sum())


In [16]:
features_windowed

,posture_median_angle_elbows,posture_median_angle_knees,posture_median_pos_wrists,posture_median_pos_elbows,posture_median_pos_ankles,posture_median_pos_knees,sym_wrists,sym_elbows,sym_knees,sym_ankles,...,range_iqr_angle_knees,effort_median_vel,effort_iqr_acc,participant,date,camera,win_start,win_stop,n_frames,filter
0,2.042888,1.594794,492.173076,545.180759,877.468954,811.629079,-0.930859,0.304234,-0.625341,0.484202,...,0.872529,34.446351,482.107496,201,2021-07-26,3,0.0,120.0,120,spline
1,1.932790,1.703898,485.498941,542.226148,901.045760,812.077504,-0.917365,0.378481,-0.892141,0.633715,...,0.727276,37.508566,580.653254,201,2021-07-26,3,60.0,180.0,120,spline
2,2.277878,2.455739,485.915464,541.746450,946.579596,835.654562,-0.855440,0.631169,-0.325761,0.744919,...,0.768850,38.054660,654.299221,201,2021-07-26,3,120.0,240.0,120,spline
3,2.330697,2.470380,489.018499,532.107178,965.928303,900.986921,-0.972036,0.815065,0.484886,0.939394,...,0.130529,24.359675,487.328298,201,2021-07-26,3,180.0,300.0,120,spline
4,2.314617,2.436993,501.406944,538.463000,939.274616,893.360957,-0.980892,0.637700,0.134043,0.706817,...,0.197128,22.764335,468.918695,201,2021-07-26,3,240.0,360.0,120,spline
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20861,0.623080,1.978203,705.055509,651.258777,543.784310,526.973007,0.999970,0.227693,-0.218926,0.999964,...,0.020835,14.885900,410.399226,252,2024-06-06,3,7020.0,7140.0,120,spline
20862,0.622913,1.979419,703.280749,650.962837,544.331014,526.980161,0.999942,-0.109917,-0.215225,0.999959,...,0.025255,13.989082,402.518003,252,2024-06-06,3,7080.0,7200.0,120,spline
20863,0.622913,2.046096,702.786507,650.775374,541.989288,526.980161,0.999921,0.129418,0.793887,0.999916,...,0.079764,14.880008,378.597510,252,2024-06-06,3,7140.0,7260.0,120,spline
20864,0.635581,2.100680,702.628175,650.714869,540.013781,527.326440,0.999970,0.324591,0.696845,0.999926,...,0.053183,15.499146,430.987092,252,2024-06-06,3,7200.0,7320.0,120,spline


In [17]:
features_windowed.columns

Index(['posture_median_angle_elbows', 'posture_median_angle_knees',
       'posture_median_pos_wrists', 'posture_median_pos_elbows',
       'posture_median_pos_ankles', 'posture_median_pos_knees', 'sym_wrists',
       'sym_elbows', 'sym_knees', 'sym_ankles', 'smooth_rms_jerk',
       'smooth_iqr_vel', 'var_entropy_wrists', 'var_entropy_elbows',
       'var_entropy_ankles', 'var_entropy_knees', 'var_entropy_angles_elbows',
       'var_entropy_angles_knees', 'range_iqr_pos', 'range_iqr_angle_elbows',
       'range_iqr_angle_knees', 'effort_median_vel', 'effort_iqr_acc',
       'participant', 'date', 'camera', 'win_start', 'win_stop', 'n_frames',
       'filter'],
      dtype='object')

In [18]:
# Count windows per recording
win_counts = (
    features_windowed
      .groupby(["participant", "date", "camera"])
      .size()
      .reset_index(name="n_windows")
)

# Count frames per recording
frame_counts = (
    keepers_long
      .groupby(["participant_id", "date", "camera"])
      .size()
      .reset_index(name="n_frames")
)

# Merge
summary = (
    frame_counts
      .merge(win_counts,
             left_on=["participant_id","date","camera"],
             right_on=["participant","date","camera"],
             how="left")
      .drop(columns="participant")
      .rename(columns={"participant_id":"participant"})
)

# Compute expected windows
fps = 30
win_len = 4 * fps   # 150 frames
step = 2 * fps      # 30 frames

summary["expected_windows"] = (
    ((summary["n_frames"] - win_len) // step + 1)
    .clip(lower=0)
)

print(summary.to_string(index=False))

 participant       date  camera  n_frames  n_windows  expected_windows
         201 2021-07-26       3      8410        139               139
         201 2021-08-19       3      7481        123               123
         201 2021-09-16       3      7486        123               123
         201 2021-10-07       3      7529        124               124
         201 2021-11-11       3      7538        124               124
         201 2021-12-16       3      7658        126               126
         202 2021-08-17       3      3686         60                60
         202 2021-09-09       3      9206        152               152
         202 2021-10-01       3      4059         66                66
         202 2021-12-10       3      7831        129               129
         202 2022-01-12       3      7652        126               126
         203 2021-09-15       3      7501        124               124
         203 2021-10-06       3      7717        127               127
      

In [19]:
features_windowed.to_csv("features_for_ioc_smooth.csv")